<a href="https://colab.research.google.com/github/KaggleStudy4th/4th_kaggle_study/blob/main/%EC%8B%A4%EC%8A%B5%EC%BD%94%EB%93%9C/20%EC%A3%BC%EC%B0%A8/DNN%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%E1%84%8B%E1%85%AF%E1%86%AB%E1%84%8C%E1%85%A1%E1%84%85%E1%85%A7%E1%86%A8%E1%84%87%E1%85%A1%E1%86%AF%E1%84%8C%E1%85%A5%E1%86%AB%E1%84%89%E1%85%A9_%E1%84%89%E1%85%A1%E1%86%BC%E1%84%90%E1%85%A2_%E1%84%91%E1%85%A1%E1%86%AB%E1%84%83%E1%85%A1%E1%86%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DNN을 이용한 원자력발전소 상태 판단

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.environ['KAGGLE_USERNAME']='jiinpp'
os.environ['KAGGLE_KEY']="b06d525a9f66198051ccb648b091778a"

In [5]:
!kaggle competitions download -c 2021-ai-w6-p1

 79% 73.0M/92.4M [00:02<00:00, 23.5MB/s]
100% 92.4M/92.4M [00:02<00:00, 35.7MB/s]


In [6]:
!unzip '*.zip'

Archive:  2021-ai-w6-p1.zip
  inflating: pca_test.csv            
  inflating: pca_train.csv           
  inflating: sample_submission.csv   
  inflating: train_label.csv         


In [23]:
X_train=pd.read_csv('pca_train.csv')
X_test=pd.read_csv('pca_test.csv')
y_train=pd.read_csv('train_label.csv').label
sample=pd.read_csv('sample_submission.csv')

In [20]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,-6.227718,-2.070960,0.674094,-0.742135,0.309149,-4.006772,-0.664624,0.533987,0.024594,-0.195856,...,-2.114192,-0.605770,0.491399,-0.166693,1.289197,-0.403863,0.238731,2.166737,-0.106487,1.486114
1,-6.081077,-1.960213,0.846482,-0.932816,-0.050208,-4.082826,-0.449032,0.647842,0.417809,0.073842,...,-1.000466,-1.175968,0.299093,-1.187048,-0.499101,0.738016,0.632842,0.850430,-2.211427,1.083040
2,-6.394956,-2.954897,1.600033,-0.479463,-1.217065,-3.986472,-0.812934,0.472509,1.536917,1.989765,...,-1.703212,0.713673,0.330699,0.036953,-0.204944,0.930597,-0.693151,-0.572443,0.835598,-0.751483
3,-6.681355,-2.055031,1.144864,-0.517872,-0.168057,-3.929032,-0.718259,0.555827,0.096472,0.724261,...,0.434870,1.571784,-1.266089,1.584173,-0.290301,0.068907,-1.148258,1.904273,0.322937,-0.318100
4,-5.716713,-2.163729,0.877930,-0.605268,0.082372,-4.261466,-0.679734,0.739369,0.490033,0.546437,...,0.626112,-0.435030,2.112485,1.101894,1.746205,-0.445612,-1.518138,0.299177,-0.020655,0.234824


In [21]:
y_train

0        136
1         90
2        112
3        165
4         73
        ... 
27666      9
27667    132
27668     18
27669     41
27670    114
Name: label, Length: 27671, dtype: int64

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [17]:
from sklearn.model_selection import train_test_split

In [24]:
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.2,random_state=1)

In [26]:
X_scaled_train=np.array(X_train)
X_scaled_val=np.array(X_val)
X_scaled_test=np.array(X_test)

In [27]:
X_train_tensor=torch.FloatTensor(X_scaled_train)
X_val_tensor=torch.FloatTensor(X_scaled_val)
X_test_tensor=torch.FloatTensor(X_scaled_test)

In [31]:
y_train_scaled=np.array(y_train)

In [32]:
y_train_tensor=torch.LongTensor(y_train_scaled)

In [54]:
y_val_scaled=np.array(y_val)
y_val_tensor=torch.FloatTensor(y_val_scaled)

In [55]:
print(X_train_tensor.shape)
print(X_val_tensor.shape)
print(y_train_tensor.shape)
print(y_val_tensor.shape)

print(X_test_tensor.shape)

torch.Size([22136, 256])
torch.Size([5535, 256])
torch.Size([22136])
torch.Size([5535])
torch.Size([13629, 256])


In [46]:
class LL(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear1=nn.Linear(256,198,bias=True)
    self.linear2=nn.Linear(198,198,bias=True)
    self.linear3=nn.Linear(198,198,bias=True)
    self.linear4=nn.Linear(198,198)
    self.relu=torch.nn.Sigmoid()

  def forward(self,x):
    x=self.linear1(x)
    x=self.linear2(x)
    x=self.linear3(x)
    x=self.linear4(x)

    return x

In [47]:
model=LL()

In [48]:
model

LL(
  (linear1): Linear(in_features=256, out_features=198, bias=True)
  (linear2): Linear(in_features=198, out_features=198, bias=True)
  (linear3): Linear(in_features=198, out_features=198, bias=True)
  (linear4): Linear(in_features=198, out_features=198, bias=True)
  (relu): Sigmoid()
)

In [64]:
loss=torch.nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=0.01)

In [50]:
for stop in range(2000):

  optimizer.zero_grad()
  hypothesis=model(X_train_tensor)
  cost=loss(hypothesis,y_train_tensor)
  cost.backward()
  optimizer.step()

  if stop%100==0:
    print(stop,cost.item())

0 5.3308844566345215
100 4.721152305603027
200 4.1865010261535645
300 3.74448299407959
400 3.432314395904541
500 3.080622434616089
600 2.831904172897339
700 2.6746432781219482
800 2.550509214401245
900 2.440390110015869
1000 2.35017466545105
1100 2.2692925930023193
1200 2.218996286392212
1300 2.157393217086792
1400 2.1061794757843018
1500 2.060152769088745
1600 2.0383834838867188
1700 1.9874244928359985
1800 1.9673151969909668
1900 1.9290368556976318


In [60]:
predict=model(X_test_tensor)
_,predicted=torch.max(predict,1)
predicted=predicted.detach().cpu().numpy()

In [61]:
pred_={"label":predicted}
submission = pd.DataFrame(data=pred_)
submission.index = [i for i in range(X_test_tensor.shape[0])]
submission.index.name = 'id'


submission.to_csv('DNN_Pred.csv', index=True) 

In [65]:
epochs=2000
from tqdm import tqdm

for i in tqdm(range(2000)):
  optimizer.zero_grad()
  hypothesis=model(X_train_tensor)
  cost=loss(hypothesis,y_train_tensor)
  cost.backward()
  optimizer.step()

  with torch.no_grad():
    val_predict=model(X_val_tensor)
    _, val_predicted=torch.max(val_predict,1)
    score=accuracy_score(val_predicted.detach().cpu(),y_val)

  if i%100==0:
    print(i,"=",score)

  0%|          | 1/2000 [00:00<18:04,  1.84it/s]

0 = 0.6066847335140018


  5%|▌         | 101/2000 [00:54<17:07,  1.85it/s]

100 = 0.6095754290876242


 10%|█         | 201/2000 [01:48<15:56,  1.88it/s]

...

 90%|█████████ | 1801/2000 [17:08<01:57,  1.69it/s]

1800 = 0.6350496838301717


 95%|█████████▌| 1901/2000 [18:08<01:09,  1.42it/s]

1900 = 0.6345076784101175


100%|██████████| 2000/2000 [19:08<00:00,  1.74it/s]


In [66]:
score

0.6341463414634146

In [67]:
predict=model(X_test_tensor)
_,predicted=torch.max(predict,1)
predicted=predicted.detach().cpu().numpy()

In [68]:
pred_={"label":predicted}
submission = pd.DataFrame(data=pred_)
submission.index = [i for i in range(X_test_tensor.shape[0])]
submission.index.name = 'id'


submission.to_csv('DNN_Pred.csv', index=True) 